This script does all the logistic regression stuff using all of the data.
It:
- goes through all assumptions for logistic regression
- alters the data wherever necessary to make things work
- creates a classifier and evaluates its performance

### Data Loading

In [2]:
import pandas as pd 
import numpy as np
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import yeojohnson

file_name = "dataset_all_days_started_30_06_23"
data_path = f'/workspaces/msc_thesis/data/final_data/{file_name}.csv'

df: pd.DataFrame = pd.read_csv(filepath_or_buffer=data_path, header=0)
df = df.sample(math.floor(len(df) * 0.1))
print(f'{len(df)=}')

pr_merged_key = 'PullRequestIsMerged'

dependent = df[pr_merged_key]
dropped_fields = [
        # Meta fields
        'ID', 'Project Name', 'Submitter ID', 'PR Number', 'Closed At',

        # dependent and control field
        pr_merged_key, 'SubmitterIsFirstTimeContributor',

        # Useless fields because they are (almost) all 0
        "DependencyEcosystemExperienceSubmitterIssueCommentCount",
        "DependencyEcosystemExperienceSubmitterIssueSubmissionCount",
        'DependencyEcosystemExperienceSubmitterPullRequestSuccessRate',
        "DependencyEcosystemExperienceSubmitterPullRequestCommentCount",
        "DependencyEcosystemExperienceSubmitterPullRequestSubmissionCount",

        "IntraProjectSubmitterIssueCommentCount",
        "IntraProjectSubmitterIssueSubmissionCount",

        "InversedDependencyEcosystemExperienceSubmitterIssueCommentCount",
        "InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount",
        'InversedDependencyEcosystemExperienceSubmitterPullRequestSuccessRate',
        "InversedDependencyEcosystemExperienceSubmitterPullRequestCommentCount",
        "InversedDependencyEcosystemExperienceSubmitterPullRequestSubmissionCount",

        "SharedExperienceIssueDiscussionParticipationByIntegratorAndSubmitter",
        'SharedExperienceIssueSubmittedByIntegratorCommentedOnBySubmitter',
        "SharedExperienceIssueSubmittedBySubmitterCommentedOnByIntegrator",

        "SharedExperiencePullRequestDiscussionParticipationByIntegratorAndSubmitter",
        "SharedExperiencePullRequestSubmittedByIntegratorCommentedOnBySubmitter",
        "SharedExperiencePullRequestSubmittedBySubmitterCommentedOnByIntegrator",
    ]
independent = df
for field in dropped_fields:
    independent = independent.drop(field, axis=1)

print(len(independent.columns))
print(independent.columns)

len(df)=182997
17
Index(['ControlIntegratedBySameUser', 'ControlPullRequestLifeTimeInMinutes',
       'ControlPullRequestHasComments', 'ControlNumberOfCommitsInPullRequest',
       'ControlPullRequestHasCommentByExternalUser',
       'ControlHasHashTagInDescription',
       'ControlIntraProjectPullRequestExperienceOfIntegrator',
       'IntraProjectSubmitterPullRequestSubmissionCount',
       'IntraProjectSubmitterPullRequestSuccessRate',
       'IntraProjectSubmitterPullRequestCommentCount',
       'EcosystemExperienceSubmitterPullRequestSuccessRate',
       'EcosystemExperienceSubmitterPullRequestSubmissionCount',
       'EcosystemExperienceSubmitterPullRequestCommentCount',
       'SharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator',
       'SharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter',
       'EcosystemExperienceSubmitterIssueSubmissionCount',
       'EcosystemExperienceSubmitterIssueCommentCount'],
      dtype='object')


### Log-odds independence

In [15]:
import json 

def show_log_odds_independence(__independent: pd.DataFrame, tested_field: str):
    # Re-running logistic regression on the original set of X and y variables
    logit_results = sm.GLM(dependent, __independent,
                           family=sm.families.Binomial()).fit()
    predicted = logit_results.predict(dependent)

    # Getting log odds values
    log_odds = np.log(predicted / (1 - predicted))

    # Visualize predictor variable vs logit values for Age
    plt.scatter(x=__independent[tested_field].values, y=log_odds)
    plt.xlabel(tested_field)
    plt.ylabel("Log-odds")
    plt.show()


def test_log_odds_independence(__independents: pd.DataFrame):
    continuous_vars = __independents.select_dtypes(include='number')
    print(f'{len(continuous_vars.columns)=}')

    # Define continuous variables
    X_lt = continuous_vars.copy()

    def x_ln_x(x):
        # 1 is added to deal with zeroes.
        return x * np.log(1 + x)

    # Add logit transform interaction terms (natural log) for continuous variables.
    ln_var_name_format = 'ln(.) x {var}'
    for var in continuous_vars:
        ln_var_name = ln_var_name_format.format(var=var)
        X_lt[ln_var_name] = X_lt[var].apply(x_ln_x)

    working_transformations = []

    for field in continuous_vars.columns:
        ln_column = ln_var_name_format.format(var=field)

        test_df = pd.DataFrame()
        test_df[field] = X_lt[field]
        test_df[ln_column] = X_lt[ln_column]


        # Removes rows with zeroes
        rows_that_are_zero = test_df.loc[test_df[field] == 0].index
        test_df = test_df.drop(rows_that_are_zero)
        test_dependent = dependent.drop(rows_that_are_zero)

        # Add constant term
        test_df = sm.add_constant(test_df, prepend=False)
        
        # Building model and fit the data.
        logit_results = sm.GLM(test_dependent,
                               test_df,
                               family=sm.families.Binomial()
                               ).fit()

        p_value = logit_results.pvalues[1]
        is_insignificant = p_value > 0.05
        significance = "insignificant" if is_insignificant else "significant"

        if is_insignificant:
            working_transformations.append(field)

        print(f'({significance}) {field}: p={p_value} (using {len(test_df)}/{len(X_lt)} entries.)')

        # Display summary results
        # print(logit_results.summary())

    print(json.dumps(working_transformations, indent=4))

# NOTE: Works for two:
# - SharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator
# - EcosystemExperienceSubmitterIssueCommentCount
test_log_odds_independence(independent)


len(continuous_vars.columns)=13
(significant) ControlPullRequestLifeTimeInMinutes: p=0.0 (using 182997/182997 entries.)
(significant) ControlNumberOfCommitsInPullRequest: p=1.59579245644442e-45 (using 182677/182997 entries.)
(significant) ControlIntraProjectPullRequestExperienceOfIntegrator: p=9.918950475822837e-199 (using 179287/182997 entries.)
(significant) IntraProjectSubmitterPullRequestSubmissionCount: p=8.901734678532992e-133 (using 129195/182997 entries.)
(significant) IntraProjectSubmitterPullRequestSuccessRate: p=2.5090634057971087e-33 (using 119303/182997 entries.)
(significant) IntraProjectSubmitterPullRequestCommentCount: p=1.0826221936684909e-52 (using 113692/182997 entries.)
(significant) EcosystemExperienceSubmitterPullRequestSuccessRate: p=8.656314461901004e-16 (using 117490/182997 entries.)
(significant) EcosystemExperienceSubmitterPullRequestSubmissionCount: p=4.2914653273240013e-206 (using 126559/182997 entries.)
(significant) EcosystemExperienceSubmitterPullRequest

In [16]:
continuous_vars = independent.select_dtypes(include='number').columns

tr_independent = independent.copy()

for field in continuous_vars:
    tr_independent[f'sqrt(1 + {field})'] =  independent[field].apply(lambda x: math.sqrt(1 + x))
    tr_independent[f'cbrt(1 + {field})'] =  independent[field].apply(lambda x: math.cbrt(1 + x))
    tr_independent[f'ln(1 + {field})'] =  independent[field].apply(lambda x: math.log(1 + x))
    tr_independent[f'log10(1 + {field})'] =  independent[field].apply(lambda x: math.log10(1 + x))
    tr_independent[f'1 / (1 + {field})'] =  independent[field].apply(lambda x: 1 / (1 + x))
    tr_independent[f'{field}^2'] =  independent[field].apply(lambda x: x ** 2)
    tr_independent[f'{field}^3'] =  independent[field].apply(lambda x: x ** 3)
    tr_independent[f'yj({field})'], _lambda = yeojohnson(independent[field])


test_log_odds_independence(tr_independent)




# NOTE: Some (3) of the variables pass after this transformation.
# - EcosystemExperienceSubmitterPullRequestSubmissionCount
# - EcosystemExperienceSubmitterPullRequestCommentCount
# - EcosystemExperienceSubmitterIssueSubmissionCount
# for field in continuous_vars:
#     tr_independent[field] = independent[field].apply(
#         lambda x: math.sqrt(1 + x))
# test_log_odds_independence(tr_independent)

# # NOTE: one of the variables pass after this transformation:
# - EcosystemExperienceSubmitterIssueSubmissionCount
# for field in continuous_vars:
#     tr_independent[field] = independent[field].apply(
#         lambda x: math.log(1 + x))
# test_log_odds_independence(tr_independent)

# NOTE: this works for one
# - EcosystemExperienceSubmitterIssueSubmissionCount
# for field in continuous_vars:
#     tr_independent[field] = independent[field].apply(
#         lambda x: math.log10(1 + x))
# test_log_odds_independence(tr_independent)

# NOTE: this works for three:
# - EcosystemExperienceSubmitterPullRequestCommentCount
# - SharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter
# - EcosystemExperienceSubmitterIssueSubmissionCount
# for field in continuous_vars:
#     tr_independent[field] = independent[field].apply(lambda x: 1/(x+1))
# test_log_odds_independence(tr_independent)

# NOTE: This works for three.
# - IntraProjectSubmitterPullRequestSubmissionCount
# - EcosystemExperienceSubmitterIssueSubmissionCount
# - EcosystemExperienceSubmitterIssueCommentCount
# for field in continuous_vars:
#     tr_independent[field] = independent[field].apply(lambda x: x**2)
# test_log_odds_independence(tr_independent)

# NOTE: This works for two:
# - SharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter   _lambda=-5.845365889174542 (reciprocal with sign preservation)
# - EcosystemExperienceSubmitterIssueSubmissionCount                         _lambda=-0.12955260658759882 (power transform with sign preservation)
# for field in continuous_vars:
#     transformed, _lambda = yeojohnson(independent[field])
#     print(f'{field=}, {_lambda=}')
#     tr_independent[field] = transformed
# test_log_odds_independence(tr_independent)

# NOTE: This does nothing.
# for field in continuous_vars:
#     tr_independent[f'squared_{field}'] = independent[field].apply(lambda x: x**2)
#     tr_independent[f'cubed_{field}'] = independent[field].apply(lambda x: x**3)
# test_log_odds_independence(tr_independent)

# NOTE: cubic root. Works for
# - EcosystemExperienceSubmitterPullRequestSubmissionCount
# - EcosystemExperienceSubmitterIssueSubmissionCount
# - IntraProjectSubmitterPullRequestCommentCount
# for field in continuous_vars:
#     tr_independent[field] = independent[field].apply(lambda x: np.cbrt(x + 1))
# test_log_odds_independence(tr_independent)

# NOTE: works for six:
# - EcosystemExperienceSubmitterPullRequestSubmissionCount
# - EcosystemExperienceSubmitterPullRequestCommentCount
# - SharedExperiencePullRequestSubmittedBySubmitterIntegratedByIntegrator
# - SharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter
# - EcosystemExperienceSubmitterIssueSubmissionCount
# - EcosystemExperienceSubmitterIssueCommentCount
# sqrt_transform = ["EcosystemExperienceSubmitterPullRequestSubmissionCount",
#                   "EcosystemExperienceSubmitterPullRequestCommentCount", "EcosystemExperienceSubmitterIssueSubmissionCount"]
# for field in sqrt_transform:
#     tr_independent[field] = independent[field].apply(
#         lambda x: math.sqrt(1 + x))
# log_transform = []
# for field in log_transform:
#     tr_independent[field] = independent[field].apply(lambda x: math.log(1 + x))
# power_transform = []
# for field in power_transform:
#     tr_independent[field] = independent[field].apply(lambda x: x ** 2)
# recip_transform = [
#     "SharedExperiencePullRequestSubmittedByIntegratorIntegratedBySubmitter"]
# for field in recip_transform:
#     tr_independent[field] = independent[field].apply(lambda x: 1 / (1+x))
# test_log_odds_independence(tr_independent)


/tmp/ipykernel_2014/3450970340.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_independent[f'cbrt(1 + {field})'] =  independent[field].apply(lambda x: math.cbrt(1 + x))
/tmp/ipykernel_2014/3450970340.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tr_independent[f'ln(1 + {field})'] =  independent[field].apply(lambda x: math.log(1 + x))
/tmp/ipykernel_2014/3450970340.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

len(continuous_vars.columns)=117


/tmp/ipykernel_2014/3912578958.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_lt[ln_var_name] = X_lt[var].apply(x_ln_x)
/tmp/ipykernel_2014/3912578958.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_lt[ln_var_name] = X_lt[var].apply(x_ln_x)
/tmp/ipykernel_2014/3912578958.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

(significant) ControlPullRequestLifeTimeInMinutes: p=0.0 (using 182997/182997 entries.)
(significant) ControlNumberOfCommitsInPullRequest: p=1.59579245644442e-45 (using 182677/182997 entries.)
(significant) ControlIntraProjectPullRequestExperienceOfIntegrator: p=9.918950475822837e-199 (using 179287/182997 entries.)
(significant) IntraProjectSubmitterPullRequestSubmissionCount: p=8.901734678532992e-133 (using 129195/182997 entries.)
(significant) IntraProjectSubmitterPullRequestSuccessRate: p=2.5090634057971087e-33 (using 119303/182997 entries.)
(significant) IntraProjectSubmitterPullRequestCommentCount: p=1.0826221936684909e-52 (using 113692/182997 entries.)
(significant) EcosystemExperienceSubmitterPullRequestSuccessRate: p=8.656314461901004e-16 (using 117490/182997 entries.)
(significant) EcosystemExperienceSubmitterPullRequestSubmissionCount: p=4.2914653273240013e-206 (using 126559/182997 entries.)
(significant) EcosystemExperienceSubmitterPullRequestCommentCount: p=2.88898235660873